Факторизационная машина

In [2]:
# библиотеки

from scipy import sparse
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import time
import os
import csv

# параметры данных
n_movies = 17770
n_users = 2649429
n = n_movies + n_users

In [ ]:
# обработка и сбор данных в файл в подходящем формате csv

combined_data_file_name = "data/combined_data_%s.txt"

target_f = open("data/processed_data.csv", "w+")
for i in {1,2,3,4}:
    print("Data file: " + str(i) + "/4")
    
    cur_movie_id = None
    with open(combined_data_file_name % i) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')

        for row in csv_reader:
            if (len(row) == 1):
                cur_movie_id = row[0][:-1]
            else:
                user_id = row[0]
                rating = row[1]
                
                target_f.write(user_id + "," + rating + "," + cur_movie_id + "\n")
target_f.close()


# деление полученного файла на 5 в соответствии с количеством фолдов,
# из-за не возможности работать с файлом целиком (нехватка памятя)

target_f1 = open("data/processed_data1.csv", "w+")
target_f2 = open("data/processed_data2.csv", "w+")
target_f3 = open("data/processed_data3.csv", "w+")
target_f4 = open("data/processed_data4.csv", "w+")
target_f5 = open("data/processed_data5.csv", "w+")
i=1
with open("data/processed_data.csv") as csv_reader:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if i == 1:
            target_f1.write(row)
            i+=1
        elif i == 2:
            target_f2.write(row)
            i+=1
        elif i == 3:
            target_f3.write(row)
            i+=1
        elif i == 4:
            target_f4.write(row)
            i+=1
        elif i == 5:
            target_f5.write(row)
            i=1
target_f1.close()
target_f2.close()
target_f3.close()
target_f4.close()
target_f5.close()

In [244]:
# создаем для каждого файла матрицу в виде: one-hot-codding фильма и записываем в файл в формате npz
# отдельно матрицу признаков и целевые значения

data = []
row = []
col = []

dataFileName = "data/processed_data%s.csv"
dataFileNameX = "data/X%s.npz"
dataFileNameY = "data/Y%s.npz"

for i in {1,2,3,4,5}:
    print("Data file: " + str(i) + "/5")
    MovieData = pd.read_csv(dataFileName % i, header = None, names = ['User_Id','Rating','Movie_Id'])
    for i in range(MovieData.shape[0]):
        # собираем данные в массивы
        data.append(1) # единица - пересечение пользователя и фильма
        data.append(1) # единица - пересечение пользователя и фильма
        data.append(MovieData['Rating'][i]) # рейтинг пользователя фильму
        row.append(i)       # на одной строке
        row.append(i)
        row.append(i)
        col.append(MovieData['Movie_Id'][i] - 1) # столбец фильма
        col.append(MovieData['User_Id'][i] + MovieData['Movie_Id'][i] - 1) # столбец пользователя
        col.append(n) # столбец целевых значений - рейтинг
            
        print(MovieData.shape)
        MovieData=''
        # разреженная матрица
        CSRMatrix = sparse.csr_matrix((data, (row, col)), dtype=np.int8, shape=(i+1, n + 1))
        data = []
        row = []
        col = []      
        #  перемешиваем данные
        CSRMatrix = shuffle(CSRMatrix)
        #  разделяем матрицу признаков и целевые значения
        X = CSRMatrix[:, :-1]
        Y = CSRMatrix[:, -1]
        sparse.save_npz(dataFileNameX % i, X)
        sparse.save_npz(dataFileNameY % i, Y)
        Y=''
        X=''
        CSRMatrix='

In [230]:
# мини-батч градиентным спуском

def gradDescentMiniBatch(ind, n, learningRate , maxIter):
    bias = np.random.random(size=(n, 3))
    w = np.random.random(size=(n, 1))
    dataFileNameX = "data/X%s.npz"
    dataFileNameY = "data/Y%s.npz"
    # проходим по всем файлом отдело, из-за нехватки памяти
    for i in {1,2,3,4,5}:
        if i != ind:
            start = time.time()
            X=sparse.load_npz(dataFileNameX % i)
            Y=sparse.load_npz(dataFileNameY % i)

            for itr in range(maxIter):
                batchSize = 131072  # размер батча 2**17
                # делим данные на батчи
                for idxB in range(int(X.shape[0] / batchSize + 1)):
                    Xbatch = X[(idxB * batchSize):((idxB * batchSize) + batchSize)]
                    yBatch = Y[(idxB * batchSize):((idxB * batchSize) + batchSize)]
                    # подсчет коэффициентов
                    batchTemp = Xbatch.dot(bias)
                    yPred = Xbatch.dot(w) + (0.5 * np.sum((batchTemp ** 2) - Xbatch.dot(bias ** 2), axis=1)).reshape(Xbatch.shape[0], 1)
                    dy = 2 * (yPred.reshape(yBatch.shape[0], 1) - yBatch.reshape(yBatch.shape[0], 1))
                    wGrad = Xbatch.T.dot(dy)
                    bGrad = (Xbatch.T.dot(batchTemp) - np.sum((Xbatch.multiply(Xbatch)), axis=0) * bias) * dy.mean() / Xbatch.shape[0]
                    w -= learningRate * wGrad
                    bias -= learningRate * bGrad

            stop = time.time()
            print(f'Data file: {i} time: {stop - start} seconds')
        
    return w, bias

In [231]:
# прогнозирование

def getPrediction(X, w, bias):
    return X.dot(w) + (0.5 * np.sum((X.dot(bias) ** 2) - X.dot(bias**2), axis=1)).reshape(X.shape[0],1)


def getPredictionTrain(ind, w, bias):
    
    dataFileNameX = "data/X%s.npz"
    prediction = []
    for i in {1,2,3,4,5}:
        if i != ind:
            X = sparse.load_npz(dataFileNameX % i)
            prediction.append(X.dot(w) + (0.5 * np.sum((X.dot(bias) ** 2) - X.dot(bias**2), axis=1)).reshape(X.shape[0],1))
            
    return np.vstack(prediction)

In [232]:
# подсчет RMSE

def getRMSE(y, yPred):
    return np.sqrt(np.mean(np.square(y - yPred)))


def getRMSETrain(ind, yPred):
    
    dataFileNameY = "data/Y%s.npz"
    y = []
    for i in {1,2,3,4,5}:
        if i != ind:
            y.append(sparse.load_npz(dataFileNameY % i))
    y = sparse.vstack(y)
    
    return np.sqrt(np.mean(np.square(y - yPred)))

In [233]:
# основная функция - кросс-валидация

def crossValidation(idx):
    
    dataFileNameX = "data/X%s.npz"
    dataFileNameY = "data/Y%s.npz"
    #данных для теста
    TestX = sparse.load_npz(dataFileNameX%idx)
    TestY = sparse.load_npz(dataFileNameY%idx)
    
    # обучение факторизационной машины 
    w,bias = gradDescentMiniBatch(idx, n, 0.0001, 3)

    # прогнозирование
    TrainPred = getPredictionTrain(idx, w, bias)
    TestPred = getPrediction(TestX, w, bias)
    # оценка
    RMSE_train = getRMSETrain(idx, TrainPred)
    RMSE_test = getRMSE(TestY, TestPred)

    print(f'RMSE train: {RMSE_train}')
    print(f'RMSE test: {RMSE_test}')

    return RMSE_train, RMSE_test

In [229]:
RMSE =[]

RMSE.append(crossValidation(1)) # параметр - номер файла, содержащий тестовые данные

Data file: 2 time: 199.7283968925476 seconds
Data file: 3 time: 197.72328329086304 seconds
Data file: 4 time: 203.19793128967285 seconds
Data file: 5 time: 195.22172689437866 seconds
RMSE train: 1.1305099079895273
RMSE test: 1.1312638020823116


In [234]:
RMSE.append(crossValidation(2))

Data file: 1 time: 188.26041746139526 seconds
Data file: 3 time: 193.58615922927856 seconds
Data file: 4 time: 197.55680298805237 seconds
Data file: 5 time: 195.46213221549988 seconds
RMSE train: 1.1313420422152807
RMSE test: 1.1324282277401179


In [235]:
RMSE.append(crossValidation(3))

Data file: 1 time: 188.06929278373718 seconds
Data file: 2 time: 188.0104057788849 seconds
Data file: 4 time: 192.82725858688354 seconds
Data file: 5 time: 204.36627984046936 seconds
RMSE train: 1.1305625800826835
RMSE test: 1.1314099887768967


In [236]:
RMSE.append(crossValidation(4))

Data file: 1 time: 167.1914553642273 seconds
Data file: 2 time: 158.4792628288269 seconds
Data file: 3 time: 158.84874200820923 seconds
Data file: 5 time: 152.47674560546875 seconds
RMSE train: 1.1301607211578755
RMSE test: 1.1306830890754191


In [237]:
RMSE.append(crossValidation(5))

Data file: 1 time: 144.21971821784973 seconds
Data file: 2 time: 147.25142884254456 seconds
Data file: 3 time: 150.87804079055786 seconds
Data file: 4 time: 153.87776160240173 seconds
RMSE train: 1.1307342724028633
RMSE test: 1.1313122579985189


In [243]:
# значение метрик RMSE
# RM = np.vstack(RMSE)
print('RMSE train')
print(np.reshape(RM[:, :-1], (1, RM.shape[0])))
print('RMSE test')
print(RM[:, -1])
print('mean train')
print(np.mean(RM[:, :-1]))
print('mean test')
print(np.mean(RM[:, -1]))
print('std train')
print(np.std(RM[:, :-1]))
print('std test')
print(np.std(RM[:, -1]))

RMSE train
[[1.13050991 1.13134204 1.13056258 1.13016072 1.13073427]]
RMSE test
[1.1312638  1.13242823 1.13140999 1.13068309 1.13131226]
mean train
1.1306619047696462
mean test
1.131419473134653
std train
0.0003878085730372459
std test
0.000564918728975363
